In [1]:
from typing import Optional, Literal, Callable, Sequence, Any
from pathlib import Path
from functools import partial
from multiprocessing import Pool
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import pydicom
from PIL import Image
import cv2
import openpyxl
cv2.setNumThreads(0)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import lightning.pytorch as pl
from torchmetrics.classification import (
    MultilabelAUROC, MultilabelAveragePrecision
)
import shutil 
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.utilities.types import OptimizerLRScheduler

from torchvision.models.resnet import resnet50

from moonshot.model import moonshot
from moonshot.image_processor import ImageProcessor
from moonshot.utils import eval_mode
from pathlib import Path
from typing import Callable, Sequence, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.resnet import resnet50
from torchvision.datasets import ImageFolder
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pydicom
import numpy as np
import os
DEVICE = torch.device('cuda')
NUM_WORKERS = 16  # поменяйте на кол-во доступных CPU

# укажите полный или относительный путь до файла с весами модели Moonshot
MOONSHOT_MODEL_FILEPATH = Path('../vision_model.pt')  # путь до модели, если она лежит в корне репозитория

# --- 1. Настройка устройства ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# --- 2. Класс Dataset для DICOM изображений ---
class DICOMDataset(Dataset):
    def __init__(self, image_paths: Sequence[Path], labels: Sequence[int], transform: Optional[Callable] = None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx: int):
        dicom_file = pydicom.dcmread(str(self.image_paths[idx]))
        img = dicom_file.pixel_array
        img = cv2.resize(img, (224, 224))  # Преобразуем размер изображения

        # Если изображение ч/б, добавим фиктивный канал
        if len(img.shape) == 2:
            img = np.expand_dims(img, axis=-1)
        
        # Преобразуем к формату albumentations
        img = np.repeat(img, 3, axis=-1).astype(np.uint8)  # Преобразуем в 3 канала
        
        if self.transform:
            img = self.transform(image=img)["image"]

        label = self.labels[idx]
        return img, label


# --- 3. Аугментации ---
transform = A.Compose([
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])


# --- 4. Модель ---
class CustomResNet(nn.Module):
    def __init__(self, num_classes: int = 1):
        super().__init__()
        self.model = resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)


# --- 5. Тренировка ---
def train_model(model, train_loader, epochs=10, lr=1e-4):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    model.to(device)
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(imgs).squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

    print("Training completed.")



c:\Users\adm\.conda\envs\moonshot\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Using device: cpu


In [2]:
data_dir = Path('D:/ds_clav_fracture_train_good/block_0000_anon/')
VINDRCXR_DIRPATH = list(data_dir.rglob("*.dcm"))  # Рекурсивный поиск DICOM файлов

destination = "D:/l"
  
for i in range(len(VINDRCXR_DIRPATH)):
    dest = shutil.move(VINDRCXR_DIRPATH[i], destination) 
print(f"Found {len(VINDRCXR_DIRPATH)} DICOM files.")
VINDRCXR_DIRPATH = "D:/l"

Found 0 DICOM files.


In [3]:
data_dir = Path(r"D:\ds_clav_fracture_train_good\block_0000_anon")
image_paths = list(data_dir.rglob("*.dcm"))  # Рекурсивный поиск DICOM файлов
print(f"Found {len(image_paths)} DICOM files.")  # Проверяем количество файлов
dicom_filepath = random.choice(list((image_paths)))
dicom = pydicom.dcmread(dicom_filepath)
image = ImageProcessor.preprocess_dicom(dicom)
image.show()

Found 0 DICOM files.


IndexError: list index out of range